In [78]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [79]:
train=pd.read_csv("train.csv")

In [80]:
train.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


LoanAmount            Loan amount in thousands

Loan_Amount_Term      Term of loan in months

Credit_History        Credit history meets guidelines

Property_Area         Urban/ Semi Urban/ Rural

Loan_Status           Loan approved (Y/N)

In [81]:
len(train)

614

In [82]:
train.dtypes

Loan_ID               object
Gender                object
Married               object
Dependents            object
Education             object
Self_Employed         object
ApplicantIncome        int64
CoapplicantIncome    float64
LoanAmount           float64
Loan_Amount_Term     float64
Credit_History       float64
Property_Area         object
Loan_Status           object
dtype: object

Preparing the data for the ML Models

In [83]:
# dealing with nans
train.isna().sum()

Loan_ID               0
Gender               13
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64

In [84]:
from sklearn.preprocessing import LabelEncoder
list_encoders=[]
from sklearn.preprocessing import StandardScaler
coulmns=None
scalar=None
means=None
def preProcessDataTrain(train):
    global means, scalar
# High Nans in gender, married, dependents, self_employed, loanAmount, loanTerms, credit_history
    NaColsCats=['Gender','Married','Dependents','Self_Employed']
#     for x in NaColsCats:
#         print(train[x].value_counts(),end='\n\n')
    train['Gender']=train['Gender'].fillna('Male')
    train['Married']=train['Married'].fillna('Yes')
    train['Dependents']=train['Dependents'].fillna('0')
    train['Self_Employed']=train['Self_Employed'].fillna('No')
#     filling the nas with mean for numeric real values
    means=train.mean()
    train=train.fillna(train.mean())
    global list_encoders
    global columns
    columns=['Gender','Married','Education','Self_Employed','Property_Area']
    for c in columns:
        cEncoder=LabelEncoder()
        train[c]=cEncoder.fit_transform(train[c].astype('str'))
        list_encoders.append(cEncoder)
    
#     coverting 3+ in train to 3
    train['Dependents']=train['Dependents'].replace(['3+'], '3')
    train['Loan_Status']=train['Loan_Status'].replace(['Y','N'],[1,0])
    # coverting dependents column to integer
    train['Dependents']=train['Dependents'].astype(str).astype(int)
    
#     no one hot encode the columns 
    col_to_one_hot=['Gender','Married','Education','Self_employed']
#     I dont think there is some need to one hot the columns, It would introduce the co linearity in the dataset
    
    train.drop(['Loan_ID'],axis=1,inplace=True)
    scaler = StandardScaler()
    train_big_num=train[['ApplicantIncome','CoapplicantIncome','LoanAmount','Loan_Amount_Term']]
    scaler.fit(train_big_num)
#     print(scaler.mean_)

    train_np=scaler.transform(train_big_num)
    
    scaled_features=pd.DataFrame(data=train_np,columns=['ApplicantIncome','CoapplicantIncome','LoanAmount','Loan_Amount_Term'])
    
    sc_feat=['ApplicantIncome','CoapplicantIncome','LoanAmount','Loan_Amount_Term']
    for sf in sc_feat:
        train[sf]=scaled_features[sf]
    return train

Let us one Hot encode some of the columns because numeric assignmnet does not make a sense

Columns are Gender, Married, Education, Self_employed



I prefer using sklearn.preprocessing.OneHotEncoder instead of pd.get_dummies This is because sklearn.preprocessing.OneHotEncoder returns an object of sklearn.preprocessing.OneHotEncoder class. We can fit this object on the training set and then use the same object to transform the test set. On the other hand, pd.get_dummies returns a dataframe with encodings based on the values in the dataframe we pass to it. This might be good for a quick analysis, but for an extended model building project where you train on training set and will be later testing on a test set, I would suggest using sklearn.preprocessing.OneHotEncoder.

In [85]:
train_pp=preProcessDataTrain(train)

In [86]:
# seeing all the lebels
for x,c in zip(list_encoders,columns):
    print('classes are ',x.classes_,' in ',c)


classes are  ['Female' 'Male']  in  Gender
classes are  ['No' 'Yes']  in  Married
classes are  ['Graduate' 'Not Graduate']  in  Education
classes are  ['No' 'Yes']  in  Self_Employed
classes are  ['Rural' 'Semiurban' 'Urban']  in  Property_Area


In [87]:
train.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [88]:
train_pp.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,1,0,0,0,0,0.072991,-0.554487,0.000000,0.279851,1.0,2,1
1,1,1,1,0,0,-0.134412,-0.038732,-0.219273,0.279851,1.0,0,0
2,1,1,0,0,1,-0.393747,-0.554487,-0.957641,0.279851,1.0,2,1
3,1,1,0,1,0,-0.462062,0.251980,-0.314547,0.279851,1.0,2,1
4,1,0,0,0,0,0.097728,-0.554487,-0.064454,0.279851,1.0,2,1


In [89]:
X=train_pp.drop(['Loan_Status'],axis=1)

In [91]:
Y=train_pp[['Loan_Status']]

In [93]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

##### Model 1

Making Logistic Regression

In [126]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

clf = LogisticRegression(penalty='l1',random_state=0,solver='liblinear',max_iter=1000,n_jobs=1)
clf.fit(x_train, np.squeeze(y_train.values))
train_preds=clf.predict(x_train)
train_acc=accuracy_score(y_train, train_preds)
print('train acc is ',train_acc)


test_preds=clf.predict(x_test)
test_acc=accuracy_score(y_test,test_preds)
print('test acc is ',test_acc)

train acc is  0.8150851581508516
test acc is  0.7980295566502463


l1 and l2 both are giving same 81% accuracy, not much accuracy but very less overfitting


Let us try SVM

In [141]:
from sklearn.svm import LinearSVC
clf = LinearSVC(tol=1e-5,penalty='l2',loss='hinge',dual=True,C=1.0,verbose=1,random_state=0,max_iter=1000)
# C,max_iter to be tuned

clf.fit(x_train, np.squeeze(y_train.values))  
print(clf.coef_)
print(clf.intercept_)

train_preds=clf.predict(x_train)
train_acc=accuracy_score(y_train, train_preds)
print('train acc is ',train_acc)


test_preds=clf.predict(x_test)
test_acc=accuracy_score(y_test,test_preds)
print('test acc is ',test_acc)

[LibLinear][[-6.44487109e-09  7.30808215e-10 -3.21991480e-09 -2.44938056e-09
  -3.29407207e-09  6.29762123e-09  7.40012711e-09 -7.91761123e-11
  -3.04565902e-10  1.99999998e+00 -4.09429552e-09]]
[-0.99999998]
train acc is  0.8150851581508516
test acc is  0.7980295566502463


Not any good results from SVM

##### Decision Trees